# Оглавление
* [Задание](#задание-2)
* [Вступление](#вступление)
* [Получаем данные о городе](#получаем-данные-о-городе)
* [Сохранение в файл](#сохранение-в-файл)
* [Загрузка из файлов](#загрузка-из-файлов)
* [Расчет самых загруженных больниц](#работа-с-данными-расчет-самых-загруженных-больниц)

# Задание 2.
Написать DAG Airflow, тасками которого будут следующие функции (город можно взять любой): 
1. Получение данных обо всех зданиях в городе и геометрии самого города с помощью библиотек osmnx и geopandas, а также сохранение данных в отдельные файлы buildings.geojson, city_geometry.geojson
2. Получение данных о больницах в городе с сохранением в отдельный файл
3. Определение количества домов, которые приходятся на каждую больницу в радиусе 500 метров (тут можно считать расстояние от точки до точки напрямую)
4. Отрисовка графика с самыми загруженными больницами и его сохранение в файл

_____
# Вступление
В этой тетрадке я буду тестировать различные функции и последовательно решать задание, перед тем, как собрать это все в один DAG. 

За город я взяла **Орлеан** во **Франции**. 

In [2]:
# Подключаем нужные библиотеки
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
from shapely.geometry import Point
import osmnx as ox
import os

______
## Получаем данные о городе

Получаем **геометрию города**:

In [3]:
city = 'France, Orlean'
city_geometry = ox.geocode_to_gdf(city)
display(city_geometry)

,geometry,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((1.87576 47.89901, 1.87576 47.899, 1....",1.875758,47.81328,1.948711,47.933558,86846510,relation,147559,47.902734,1.908607,boundary,administrative,16,0.662141,city,Orléans,"Orléans, Loiret, Centre-Val de Loire, Metropol..."


Получаем **данные обо всех зданиях** в городе.

In [4]:
buildings_gdf = ox.features_from_place(city, {'building': True})
display(buildings_gdf.head(3))
display(buildings_gdf.describe())

geometry   building contact:city  \
element id                                                             
node    4428949791     POINT (1.88 47.89886)  dormitory      Orléans   
        6458548716  POINT (1.90296 47.91474)    service          NaN   
        7549263591  POINT (1.89344 47.91504)        yes          NaN   

                   contact:housenumber      contact:phone contact:postcode  \
element id                                                                   
node    4428949791                   1  +33 3 20 12 25 89            45000   
        6458548716                 NaN                NaN              NaN   
        7549263591                 NaN                NaN              NaN   

                       contact:street           name  \
element id                                             
node    4428949791  Place De L'europe  Twenty Campus   
        6458548716                NaN            NaN   
        7549263591                NaN            NaN   

                                                  official_name  \
element id                                                        
node    4428949791  Twenty Campus Orléans - Résidence Étudiante   
        6458548716                                          NaN   
        7549263591                                          NaN   

                                     opening_hours  ...  \
element id                                          ...   
node    4428949791  Mo-Fr 09:00-12:00, 14:00-18:00  ...   
        6458548716                             NaN  ...   
        7549263591                             NaN  ...   

                   check_date:tactile_paving name:fr building:min_levels  \
element id                                                                 
node    4428949791                       NaN     NaN                 NaN   
        6458548716                       NaN     NaN                 NaN   
        7549263591                       NaN     NaN                 NaN   

                   lockable frequency architect:wikidata house full_name  \
element id                                                                 
node    4428949791      NaN       NaN                NaN   NaN       NaN   
        6458548716      NaN       NaN                NaN   NaN       NaN   
        7549263591      NaN       NaN                NaN   NaN       NaN   

                   live_music currency:EUR  
element id                                  
node    4428949791        NaN          NaN  
        6458548716        NaN          NaN  
        7549263591        NaN          NaN  

[3 rows x 257 columns]

,geometry,building,contact:city,contact:housenumber,contact:phone,contact:postcode,contact:street,name,official_name,opening_hours,...,check_date:tactile_paving,name:fr,building:min_levels,lockable,frequency,architect:wikidata,house,full_name,live_music,currency:EUR
count,33723,33723,16,16,5,16,2,653,5,37,...,1,1,1,1,2,1,15,1,1,1
unique,33723,55,1,12,5,2,2,596,5,32,...,1,1,1,1,1,1,2,1,1,1
top,POINT (1.8972891 47.8843577),yes,Orléans,2,+33 3 20 12 25 89,45000,Place De L'europe,Campo Santo,Twenty Campus Orléans - Résidence Étudiante,24/7,...,2025-03-23,chr orlans,0,yes,50,Q122730688,detached,Institut de Chimie Organique et Analytique,yes,yes
freq,1,28139,16,2,1,14,1,9,1,5,...,1,1,1,1,2,1,12,1,1,1


Получим **данные о больницах** в городе. Это можно получить двумя способами, проверим оба и сравним.
1. Через `'building': 'hospital'`

In [5]:
# hospitals = buildings_gdf[buildings_gdf.building == 'hospital']  # можно вытащить из датафрейма всех зданий
hospitals = ox.features_from_place(city, tags = {'building': 'hospital'})  # а можно еще раз обратиться к API
display(hospitals.head(3))
display(hospitals.describe())

geometry  building  \
element  id                                                                     
relation 1592266  POLYGON ((1.89736 47.90092, 1.89778 47.90096, ...  hospital   
         7205438  POLYGON ((1.92015 47.83457, 1.92015 47.83464, ...  hospital   
         7206143  POLYGON ((1.92288 47.83782, 1.92309 47.83809, ...  hospital   

                 source building:levels building:part  \
element  id                                             
relation 1592266    NaN             NaN           NaN   
         7205438    NaN               4           NaN   
         7206143    NaN               4           NaN   

                                       name operator building:min_level layer  \
element  id                                                                     
relation 1592266  Ancien Hôpital Hôtel-Dieu      NaN                NaN   NaN   
         7205438                 Point Vert     CHUO                NaN     1   
         7206143                 Point Bleu     CHUO                NaN     1   

                 addr:city  ... description emergency healthcare:speciality  \
element  id                 ...                                               
relation 1592266       NaN  ...         NaN       NaN                   NaN   
         7205438   Orléans  ...         NaN       yes                   NaN   
         7206143       NaN  ...         NaN       NaN                   NaN   

                 healthcare:speciality:FR opening_hours building:level  \
element  id                                                              
relation 1592266                      NaN           NaN            NaN   
         7205438                      NaN           NaN            NaN   
         7206143                      NaN           NaN            NaN   

                 building:colour          type  \
element  id                                      
relation 1592266             NaN  multipolygon   
         7205438         #ccffb3  multipolygon   
         7206143             NaN  multipolygon   

                                                              image  \
element  id                                                           
relation 1592266                                                NaN   
         7205438  https://media.ouest-france.fr/v1/pictures/MjAy...   
         7206143                                                NaN   

                        mapillary  
element  id                        
relation 1592266              NaN  
         7205438  655412022438709  
         7206143  362343379097212  

[3 rows x 24 columns]

,geometry,building,source,building:levels,building:part,name,operator,building:min_level,layer,addr:city,...,description,emergency,healthcare:speciality,healthcare:speciality:FR,opening_hours,building:level,building:colour,type,image,mapillary
count,21,21,2,19,2,12,18,6,9,2,...,1,2,1,1,1,1,3,7,1,4
unique,21,1,2,7,1,12,2,4,3,1,...,1,2,1,1,1,1,3,1,1,4
top,"POLYGON ((1.897365 47.900922, 1.8977764 47.900...",hospital,cadastre-dgi-fr source : Direction Générale de...,4,hospital,Ancien Hôpital Hôtel-Dieu,CHUO,0,1,Orléans,...,Centre de Régulation et de Réception des Appel...,yes,emergency,SAMU,24/7,2,#ccffb3,multipolygon,https://media.ouest-france.fr/v1/pictures/MjAy...,655412022438709
freq,1,21,1,6,2,1,16,2,6,2,...,1,1,1,1,1,1,1,7,1,1


2. Через `'amentiy': 'hospital'`

In [6]:
hospitals1 = ox.features_from_place(city, tags = {'amenity': 'hospital'})
display(hospitals1.head(3))
display(hospitals1.describe())

geometry  \
element id                                                              
node    7858296123                           POINT (1.87845 47.89968)   
        9732244602                           POINT (1.90395 47.90461)   
way     355306681   POLYGON ((1.91738 47.83683, 1.91754 47.83705, ...   

                     amenity                fax healthcare  \
element id                                                   
node    7858296123  hospital       +33238867767   hospital   
        9732244602  hospital                NaN   hospital   
way     355306681   hospital  +33 2 38 51 49 50   hospital   

                                                          name  \
element id                                                       
node    7858296123                Centre de jour Marcel Proust   
        9732244602               Clinique laser vision Orléans   
way     355306681   Centre Hospitalier Universitaire d'Orléans   

                                phone ref:FR:FINESS    ref:FR:SIRET  \
element id                                                            
node    7858296123       +33238721908     450012620  26450004200090   
        9732244602  +33 2 42 04 04 20           NaN             NaN   
way     355306681   +33 2 38 51 44 44     450002613             NaN   

                                                               source  \
element id                                                              
node    7858296123  Le ministère des solidarités et de la santé - ...   
        9732244602                                                NaN   
way     355306681                                                 NaN   

                   type:FR:FINESS  ... wheelchair          addr:street  \
element id                         ...                                   
node    7858296123            292  ...        NaN                  NaN   
        9732244602            NaN  ...        yes                  NaN   
way     355306681             101  ...        NaN  Avenue de l'Hôpital   

                                alt_name contact:facebook contact:instagram  \
element id                                                                    
node    7858296123                   NaN              NaN               NaN   
        9732244602                   NaN              NaN               NaN   
way     355306681   Hôpital de la Source       chrorleans       chu.orleans   

                   contact:twitter emergency operator:type short_name  \
element id                                                              
node    7858296123             NaN       NaN           NaN        NaN   
        9732244602             NaN       NaN           NaN        NaN   
way     355306681      chr_orleans       yes        public       CHRO   

                     wikidata  
element id                     
node    7858296123        NaN  
        9732244602        NaN  
way     355306681   Q30739541  

[3 rows x 28 columns]

,geometry,amenity,fax,healthcare,name,phone,ref:FR:FINESS,ref:FR:SIRET,source,type:FR:FINESS,...,wheelchair,addr:street,alt_name,contact:facebook,contact:instagram,contact:twitter,emergency,operator:type,short_name,wikidata
count,4,4,2,4,3,3,2,1,1,2,...,1,1,1,1,1,1,1,1,1,1
unique,4,1,2,1,3,3,2,1,1,2,...,1,1,1,1,1,1,1,1,1,1
top,POINT (1.8784546 47.8996762),hospital,+33238867767,hospital,Centre de jour Marcel Proust,+33238721908,450012620,26450004200090,Le ministère des solidarités et de la santé - ...,292,...,yes,Avenue de l'Hôpital,Hôpital de la Source,chrorleans,chu.orleans,chr_orleans,yes,public,CHRO,Q30739541
freq,1,4,1,4,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


Удалим из всех датафреймов индекс `element` и перенесем индекс `id` в качестве обычного столбца.

In [7]:
buildings_gdf = buildings_gdf.reset_index(level='element', drop=True)
buildings_gdf = buildings_gdf.reset_index(level='id')
hospitals = hospitals.reset_index(level='element', drop=True)
hospitals = hospitals.reset_index(level='id')
hospitals1 = hospitals1.reset_index(level='element', drop=True)
hospitals1 = hospitals1.reset_index(level='id')

In [8]:
buildings_gdf.head(2)

,id,geometry,building,contact:city,contact:housenumber,contact:phone,contact:postcode,contact:street,name,official_name,...,check_date:tactile_paving,name:fr,building:min_levels,lockable,frequency,architect:wikidata,house,full_name,live_music,currency:EUR
0,4428949791,POINT (1.88 47.89886),dormitory,Orléans,1,+33 3 20 12 25 89,45000,Place De L'europe,Twenty Campus,Twenty Campus Orléans - Résidence Étudiante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6458548716,POINT (1.90296 47.91474),service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Теперь **сравним датафреймы** `hospitals` и `hospitals1`. В первом датафрейме зданий больше. Сравним полученные данные с помощью **визуализации**.

Красным обозначены больницы (полигоны), полученные способом через `building`, а синим через `amenity`.

In [9]:
# Карта Орлеана
f = folium.Figure(width=750, height=500)
basemap = city_geometry.explore(tooltip=False, color='grey')
basemap.add_to(f)

# Добавляем больницы
hospitals1.explore(tooltip=['name', 'id'], color='blue', m=basemap)
hospitals.explore(tooltip=['name', 'id'], color='red', m=basemap)

Что мы можем сказать:
1. Хоть и в датафрейме `hospitals`, полученным способом через building (первый датафрейм), зданий больше, почти все они относятся к одному и тому же комплексу на юге города. Правильнее считать все эти здания одной больницей, как это сделано во втором датафрейме `hospitals1`. 
2. В `hospitals` нет трех других больниц, которые появляются в `hospitals1`. При этом у одной из этих больниц нет названия, ее мы рассмотрим отдельно.
3. В `hospitals1` тоже нет двух больниц, которые появляются в `hospitals`. При этом у одной из них тоже нет названия.


Рассмотрим здания, у которых нет названия - существуют ли они и являюся ли больницами. Для этого я буду использовать гугл карты. 


In [10]:
hospitals1[hospitals1.name.isnull()]

,id,geometry,amenity,fax,healthcare,name,phone,ref:FR:FINESS,ref:FR:SIRET,source,...,wheelchair,addr:street,alt_name,contact:facebook,contact:instagram,contact:twitter,emergency,operator:type,short_name,wikidata
3,1235184394,"POLYGON ((1.91846 47.90656, 1.91915 47.90658, ...",hospital,NaN,hospital,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Это здание, судя по гугл картам, является домом престарелых.

In [11]:
hospitals[hospitals.id == 113997817]

,id,geometry,building,source,building:levels,building:part,name,operator,building:min_level,layer,...,description,emergency,healthcare:speciality,healthcare:speciality:FR,opening_hours,building:level,building:colour,type,image,mapillary
7,113997817,"POLYGON ((1.89778 47.90012, 1.89712 47.90008, ...",hospital,cadastre-dgi-fr source : Direction Générale de...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Это здание существует, но невозможно понять, какую функцию оно выполняет, и какие организации в нем находятся. Никакой дополнительной информации об этом здании нет, поэтому я делаю вывод, что оно, возможно, относится к зданию выше.

**Вывод:**
- Если необходимо создать **универсальную** программу, которая отображает все больницы (и в дальнейшем работает с ними) для **произвольного** города, то я бы воспользовалась только свойством `'amenity': 'hospital'`. На карте такие здания помечаются "плюсиком" и с большей вероятностью являются легитимными больницами.
- Если требуется проводить **глубокий анализ** для **конкретного** города, то можно **объединить** данные из разных источников, используя `'amenity': 'hospital'`, `'building': 'hospital'` (при этом исключив пересекающиеся), а также обогатить данными из других источников, чтобы составить более полную картину.
- Для зданий, от которых известен лишь их полигон и нет названия, есть два варианта. Их можно либо отбрасывать, либо наоборот включать.

Для данной задачи я возьму датафрейм `'hospitals1'` (отобранный по свойству `'amenity': 'hospital'`) и не буду добавлять никаких других данных. Здания без названия исключать не буду.

____
# Сохранение в файл
**Сохраним** полученные данные в виде файлов формата .geojson

In [12]:
# Путь к папке для сохранения файлов
OUTPUT_PATH = os.path.join(os.getcwd(), 'output')

city_geometry.to_file(os.path.join(OUTPUT_PATH, 'city_geometry.geojson'))
buildings_gdf.to_file(os.path.join(OUTPUT_PATH, 'buildings.geojson'))
hospitals1.to_file(os.path.join(OUTPUT_PATH, 'hospitals.geojson'))

____
# Загрузка из файлов

Загружаем данные из файлов.

In [13]:
city_geometry_copy = gpd.read_file(os.path.join(OUTPUT_PATH, 'city_geometry.geojson'))
buildings_gdf_copy = gpd.read_file(os.path.join(OUTPUT_PATH, 'buildings.geojson'))
hospitals_copy = gpd.read_file(os.path.join(OUTPUT_PATH, 'hospitals.geojson'))

Сравниваем на соответствие изначальным даным

In [14]:
display(city_geometry)
display(city_geometry_copy)

,geometry,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"POLYGON ((1.87576 47.89901, 1.87576 47.899, 1....",1.875758,47.81328,1.948711,47.933558,86846510,relation,147559,47.902734,1.908607,boundary,administrative,16,0.662141,city,Orléans,"Orléans, Loiret, Centre-Val de Loire, Metropol..."


,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name,geometry
0,1.875758,47.81328,1.948711,47.933558,86846510,relation,147559,47.902734,1.908607,boundary,administrative,16,0.662141,city,Orléans,"Orléans, Loiret, Centre-Val de Loire, Metropol...","POLYGON ((1.87576 47.89901, 1.87576 47.899, 1...."


Столбец `geometry` переместился в конец, но в остальном кажется, что данные остались такими же. Проверка показывает, что также слека изменились типы данных (int64 на int32)

In [15]:
# Сравним столбцы geometry
city_geometry['geometry'].equals(city_geometry_copy['geometry'])

True

In [16]:
# Сравним все остальные столбцы кроме столбца geometry

from pandas.testing import assert_frame_equal

df1 = city_geometry.loc[:,'bbox_west':'display_name']
df2 = city_geometry_copy.loc[:,'bbox_west':'display_name']

# display(df1.equals(df2))  # Возвращает False
# assert_frame_equal(df1, df2)  # Возващает ошибку 
    # Attributes of DataFrame.iloc[:, 4] (column name="place_id") are different

    # Attribute "dtype" are different
    # [left]:  int64
    # [right]: int32

# Поменяем все int32 на int64 для этой проверки:
assert_frame_equal(df1, df2.astype({"place_id": 'int64', "osm_id" : 'int64', "place_rank": 'int64'}))  # Ошибок больше не возникает
df1.equals(df2.astype({"place_id": 'int64', "osm_id" : 'int64', "place_rank": 'int64'}))  # Возвращает True

True

Получается, данные идентичны, просто int64 меняется на int32, а столбец `geometry` поменял свое положение. Для нашей задачи это не критично. Теперь сделаем то же самое для двух других файлов.

In [17]:
display(hospitals1.head(3))
display(hospitals_copy.head(3))

,id,geometry,amenity,fax,healthcare,name,phone,ref:FR:FINESS,ref:FR:SIRET,source,...,wheelchair,addr:street,alt_name,contact:facebook,contact:instagram,contact:twitter,emergency,operator:type,short_name,wikidata
0,7858296123,POINT (1.87845 47.89968),hospital,+33238867767,hospital,Centre de jour Marcel Proust,+33238721908,450012620,26450004200090,Le ministère des solidarités et de la santé - ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9732244602,POINT (1.90395 47.90461),hospital,NaN,hospital,Clinique laser vision Orléans,+33 2 42 04 04 20,NaN,NaN,NaN,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,355306681,"POLYGON ((1.91738 47.83683, 1.91754 47.83705, ...",hospital,+33 2 38 51 49 50,hospital,Centre Hospitalier Universitaire d'Orléans,+33 2 38 51 44 44,450002613,NaN,NaN,...,NaN,Avenue de l'Hôpital,Hôpital de la Source,chrorleans,chu.orleans,chr_orleans,yes,public,CHRO,Q30739541


,id,amenity,fax,healthcare,name,phone,ref:FR:FINESS,ref:FR:SIRET,source,type:FR:FINESS,...,addr:street,alt_name,contact:facebook,contact:instagram,contact:twitter,emergency,operator:type,short_name,wikidata,geometry
0,7858296123,hospital,+33238867767,hospital,Centre de jour Marcel Proust,+33238721908,450012620,26450004200090,Le ministère des solidarités et de la santé - ...,292,...,None,None,None,None,None,None,None,None,None,POINT (1.87845 47.89968)
1,9732244602,hospital,None,hospital,Clinique laser vision Orléans,+33 2 42 04 04 20,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (1.90395 47.90461)
2,355306681,hospital,+33 2 38 51 49 50,hospital,Centre Hospitalier Universitaire d'Orléans,+33 2 38 51 44 44,450002613,None,None,101,...,Avenue de l'Hôpital,Hôpital de la Source,chrorleans,chu.orleans,chr_orleans,yes,public,CHRO,Q30739541,"POLYGON ((1.91738 47.83683, 1.91754 47.83705, ..."


In [18]:
display(buildings_gdf.head(3))
display(buildings_gdf_copy.head(3))

,id,geometry,building,contact:city,contact:housenumber,contact:phone,contact:postcode,contact:street,name,official_name,...,check_date:tactile_paving,name:fr,building:min_levels,lockable,frequency,architect:wikidata,house,full_name,live_music,currency:EUR
0,4428949791,POINT (1.88 47.89886),dormitory,Orléans,1,+33 3 20 12 25 89,45000,Place De L'europe,Twenty Campus,Twenty Campus Orléans - Résidence Étudiante,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6458548716,POINT (1.90296 47.91474),service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7549263591,POINT (1.89344 47.91504),yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,id,building,contact:city,contact:housenumber,contact:phone,contact:postcode,contact:street,name,official_name,opening_hours,...,name:fr,building:min_levels,lockable,frequency,architect:wikidata,house,full_name,live_music,currency:EUR,geometry
0,4428949791,dormitory,Orléans,1,+33 3 20 12 25 89,45000,Place De L'europe,Twenty Campus,Twenty Campus Orléans - Résidence Étudiante,"Mo-Fr 09:00-12:00, 14:00-18:00",...,None,None,None,None,None,None,None,None,None,POINT (1.88 47.89886)
1,6458548716,service,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (1.90296 47.91474)
2,7549263591,yes,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (1.89344 47.91504)


Аналогично, то же самое, столбец `geometry` переместился в конец и слегка изменились типы данных.

____
# Работа с данными. Расчет самых загруженных больниц
> Задание:
Определение количества домов, которые приходятся на каждую больницу в радиусе 500 метров (тут можно считать расстояние от точки до точки напрямую)

Для следующей работы с данными я буду использовать датафреймы с суффиксом `_copy` (это те, которые были вытащены из файлов), т.к. именно в таком виде они будут использоваться в DAG-е.

In [19]:
# Преобразуем CRS в проекцию, которая сохраняет расстояния
# Мы используем метод to_crs() для преобразования геометрий зданий и больниц в проекцию EPSG:3857 (Web Mercator), 
# это проекция, которая используется в большинстве веб-карт и она подходит для расчетов расстояний.
buildings_gdf_copy1 = buildings_gdf_copy.to_crs(epsg=3857)  # Web Mercator
hospitals_copy1 = hospitals_copy.to_crs(epsg=3857)  # Web Mercator

# Список с количеством зданий для каждой больницы
buildings_per_hospital = []

# Переберем все больницы:
for _, hospital in hospitals_copy1.iterrows():
    # Считаем расстояние до всех зданий
    distances = buildings_gdf_copy1.geometry.distance(hospital.geometry)
    # Считаем количество зданий в радиусе 500 метров
    buildings_count = (distances <= 500).sum()
    # Сохраняем в список
    buildings_per_hospital.append({
        'id': hospital.id,
        'buildings_count': buildings_count
    })

# Преобразуем список в датафрейм для более красивого отображения
buildings_per_hospital = pd.DataFrame(buildings_per_hospital)
display(buildings_per_hospital)

# Добавим информацию о количестве зданий в изначальный датафрейм больниц (в этом датафрейме геометрия находится в проекции Web Mercator)
hospitals_final = pd.merge(hospitals_copy1, buildings_per_hospital, on=['id'])
display(hospitals_final)

,id,buildings_count
0,7858296123,307
1,9732244602,965
2,355306681,513
3,1235184394,1014


,id,amenity,fax,healthcare,name,phone,ref:FR:FINESS,ref:FR:SIRET,source,type:FR:FINESS,...,alt_name,contact:facebook,contact:instagram,contact:twitter,emergency,operator:type,short_name,wikidata,geometry,buildings_count
0,7858296123,hospital,+33238867767,hospital,Centre de jour Marcel Proust,+33238721908,450012620,26450004200090,Le ministère des solidarités et de la santé - ...,292,...,None,None,None,None,None,None,None,None,POINT (209108.61 6090180.728),307
1,9732244602,hospital,None,hospital,Clinique laser vision Orléans,+33 2 42 04 04 20,None,None,None,None,...,None,None,None,None,None,None,None,None,POINT (211946.21 6091000.432),965
2,355306681,hospital,+33 2 38 51 49 50,hospital,Centre Hospitalier Universitaire d'Orléans,+33 2 38 51 44 44,450002613,None,None,101,...,Hôpital de la Source,chrorleans,chu.orleans,chr_orleans,yes,public,CHRO,Q30739541,"POLYGON ((213442.032 6079751.648, 213459.921 6...",513
3,1235184394,hospital,None,hospital,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((213561.512 6091323.437, 213638.578 6...",1014


Также рассчитаем **расстояние от каждого здания до ближайшей больницы**, т.к. это будет удобно для дальнейшей визуализации.

In [20]:
# Создаем список для хранения расстояний
distances_to_nearest_hospital = []

for _, building in buildings_gdf_copy1.iterrows():
    # Вычисляем расстояния до всех больниц
    distances = hospitals_copy1.geometry.distance(building.geometry)
    # Находим минимальное расстояние
    min_distance = distances.min()
    # Находим id больницы, докоторой было минимальное расстояние
    min_idx = distances.idxmin()
    nearest_id = hospitals_copy1.loc[min_idx, 'id']
    # Сохраняем в список
    distances_to_nearest_hospital.append({
        'id': building.id,
        'hospital_id': nearest_id,
        'distance_to_nearest_hospital': min_distance
    })

# Преобразуем список в датафрейм
distances_to_nearest_hospital = pd.DataFrame(distances_to_nearest_hospital)

# Добавим информацию в изначальный датафрейм зданий (в этом датафрейме геометрия находится в ИЗНАЧАЛЬНОЙ проекции)
buildings_final = pd.merge(buildings_gdf_copy, distances_to_nearest_hospital, on=['id'])
display(buildings_final.head())

,id,building,contact:city,contact:housenumber,contact:phone,contact:postcode,contact:street,name,official_name,opening_hours,...,lockable,frequency,architect:wikidata,house,full_name,live_music,currency:EUR,geometry,hospital_id,distance_to_nearest_hospital
0,4428949791,dormitory,Orléans,1,+33 3 20 12 25 89,45000,Place De L'europe,Twenty Campus,Twenty Campus Orléans - Résidence Étudiante,"Mo-Fr 09:00-12:00, 14:00-18:00",...,None,None,None,None,None,None,None,POINT (1.88 47.89886),7858296123,219.212805
1,6458548716,service,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,POINT (1.90296 47.91474),9732244602,1685.380843
2,7549263591,yes,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,POINT (1.89344 47.91504),9732244602,2088.999999
3,7944677540,yes,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,POINT (1.90978 47.89858),9732244602,1194.012318
4,7944677541,yes,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,POINT (1.90974 47.89862),9732244602,1186.130952


______
# Визуализация

1. Визуализируем базовую карту - геометрия города Орлеан.

In [21]:
f = folium.Figure(width=750, height=500)
basemap = city_geometry.explore(tooltip=False, color='grey')
basemap.add_to(f)
basemap

2. Изображаем на этой карте здания, больницы а также круг вокруг каждой больницы, который символизирует радиус в 500 метров. 
   
   **Больницы** выделены красным цветом, при наведении показывается id больницы и ее название.   
   **Область в 500 метров** вокруг больниц - синим цветом, при наведении показывается количество зданий в этом радиусе.  
   **Здания** - цветом по тепловой карте (чем больше расстояние, тем темнее цвет), при наведении показывает расстояние до ближайшей больницы.

In [ ]:
# 1. Добавляем здания на карту
buildings_final.explore(tooltip=['distance_to_nearest_hospital'],
                        column='distance_to_nearest_hospital',  # колонка для окраски по тепловой карте
                        cmap='magma_r',  # тепловая карта
                        m=basemap)

# 2. Добавляем круг вокруг больниц, который определяет радиус в 500 метров
for _, hospital in hospitals_final.iterrows():
    # Создаем буфер (координаты должны быть в проекции Web Mercator, в hospitals_final они так и хранятся)
    buffer_geom = hospital.geometry.buffer(500)  # 500 метров

    # Переводим буфер обратно в систему координат, в которой рисует Folium
    buffer_wgs84 = gpd.GeoSeries([buffer_geom], crs='EPSG:3857').to_crs(epsg=4326).iloc[0]

    # Создаем текст подсказки при наведении 
    tooltip = f'{hospital.buildings_count} зданий в радиусе 500 метров от больницы'

    # Добавляем буфер на карту
    folium.GeoJson(buffer_wgs84, tooltip=tooltip, style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'fillOpacity': 0.4,
        'weight': 1,
    }).add_to(basemap)

# 3. Добавляем больницы
hospitals_copy.explore(tooltip=['name', 'id'], color='red', m=basemap)

# Отображаем получившуюся карту
basemap

3. Сохраняем получившуюся карту в файл

In [23]:
basemap.save(os.path.join(OUTPUT_PATH, 'orlean_map.html'))